In [4]:
import pandas as pd
import numpy as np
import csv
import dask
import dask.dataframe as dd
import psycopg2

# settings
#datapath = '/Users/mirkoknoche/Google Drive/neuefische/projects/MIMIC/data/'
datapath = '/Users/mirkoknoche/MIMIC/'
gitpath = '/Users/mirkoknoche/Google Drive/neuefische/projects/MIMIC/'
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,}'.format

# database
conn_string = open(gitpath + 'intern/DB_connection.txt', 'r', encoding='UTF-8').read()

In [ ]:
import psycopg2
conn_string = open(gitpath + 'intern/DB_connection.txt', 'r', encoding='UTF-8').read()
conn = eval('psycopg2.connect({})'.format(conn_string.strip()))
sql = """select * from cpt 
        where subject_id in (select subject_id from diagnoses_icd di where ICD9_CODE = '51881')"""
df_sql = pd.read_sql(sql, conn)

conn.close()

# Datenbank

In [2]:
import psycopg2
gitpath = '/Users/mirkoknoche/Google Drive/neuefische/projects/MIMIC/'
conn_string = open(gitpath + 'intern/DB_connection.txt', 'r', encoding='UTF-8').read()
conn = eval('psycopg2.connect({})'.format(conn_string.strip()))

In [5]:
sql = """select * from vw_t_chartevents LIMIT 1000"""
df_sql = pd.read_sql(sql, conn)

In [6]:
df_sql.head()

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,321515695,"90,917.0","137,240.0","236,280.0","223,902.0",2162-11-02 12:00:00,2162-11-02 12:04:00,"14,576.0",Normal,nan,None,0.0,0.0,None,None
1,321515696,"90,917.0","137,240.0","236,280.0","223,911.0",2162-11-02 12:00:00,2162-11-02 12:04:00,"14,576.0",Purposeful,nan,None,0.0,0.0,None,None
2,321515697,"90,917.0","137,240.0","236,280.0","223,916.0",2162-11-02 12:00:00,2162-11-02 12:04:00,"14,576.0",Verbal,nan,None,0.0,0.0,None,None
3,321515698,"90,917.0","137,240.0","236,280.0","223,917.0",2162-11-02 12:00:00,2162-11-02 12:04:00,"14,576.0",Normal strength,nan,None,0.0,0.0,None,None
4,321515699,"90,917.0","137,240.0","236,280.0","223,918.0",2162-11-02 12:00:00,2162-11-02 12:04:00,"14,576.0",Moves on Bed,nan,None,0.0,0.0,None,None


In [ ]:
df_sql.info()

In [ ]:
conn.close()

# Tests

## Change headerlines

In [ ]:
with open(datapath + 'liste_patienten.csv') as f:
    reader = csv.reader(f)
    data = list(reader)
    data = [x for x in data[0]]

In [ ]:
data

In [ ]:
cz = 7000000
chunk = pd.read_csv(datapath + 'chartevents.csv', nrows=100000) #, chunksize=cz)

In [ ]:
chunk.sample(10)

In [ ]:
for i, df in enumerate(chunk):
    print('Durchlauf ', i+1, 'gestarted.')
    df = df.loc[df.HADM_ID].isin(data)
    df.to_csv('{}chunk/ARDS{}.csv'.format(datapath, i), index=False)
    #df.loc[~df.HADM_ID].isin(data).to_csv('{}chunk/Other_{}.csv'.format(datapath, i), index=False)
    #df.to_csv('{}chunk/file{}.csv'.format(datapath, i), index=False)

## Read csv header

In [ ]:
df_csv = pd.read_csv(datapath + 'ards_aj.csv.gz', compression='gzip', usecols=[1], header=None)

In [ ]:
lst_count.append(len(df_csv))

In [ ]:
print(len(lst_count))
print(f'{sum(lst_count):,}')


In [ ]:
sql = """select row_id from chartevents"""
df_sql = pd.read_sql(sql, conn)


In [ ]:
len(df_sql)

In [ ]:
df_chart = pd.read_csv(datapath + 'chartevents.csv', usecols=[0]) #, chunksize=cz)

# inputevents_cv

Inputs are any fluids which have been administered to the patient: such as oral or tube feedings or intravenous solutions containing medications. 

Inputs exist in two separate tables: INPUTEVENTS_CV and INPUTEVENTS_MV. INPUTEVENTS_CV contains CareVue inputs, while INPUTEVENTS_MV contains Metavision inputs. Results from these tables can be unioned as observations are not duplicated across tables. For CareVue data, the rate and volume will be asynchronous, and only the CHARTTIME will be available. For rates, the CHARTTIME will correspond to a start time (when the drug was set to that rate). For volumes, the CHARTTIME will correspond to an end time.

| column | Description | Type |
|:-------|:------------|------|
| ROW_ID | ID from Database | numeric |
| SUBJECT_ID | Unique ID of a patient | numeric |
| HADM_ID | Unique ID of a patient hospital stay | numeric |
| ICUSTAY_ID | Unique ID of a patient ICU stay | numeric |
| CHARTTIME | time at which the measurement was charted at the bedside | datetime |
| ITEMID | Identifier for a single measurement type D_ITEMS.csv | numeric |
| AMOUNT | amount of a drug or substance administered to the patient either between the STARTTIME and ENDTIME | numeric |

In [ ]:
c = 1000000
df_input_cv = pd.read_csv(datapath + 'INPUTEVENTS_CV.csv', nrows=10000)

In [ ]:
sql = """select * from inputevents_cv 
        where subject_id in (select subject_id from diagnoses_icd di where ICD9_CODE = '51881')
        LIMIT 100000"""
df_input_cv = pd.read_sql(sql, conn)

In [ ]:
df_input_cv.shape

In [ ]:
df_input_cv.info()

We can see that already in the first 100000 rows some HADM_ID's are missing. Let us show how many, if we import only this column.

In [ ]:
sql = """select hadm_id from inputevents_cv """
df_test = pd.read_sql(sql, conn)

In [ ]:
df_test.shape

In [ ]:
df_test.isna().sum()

In [ ]:
print('Missing HADM_ID in percent: {:.2f}%'.format(float(df_test.isna().sum()) / df_test.shape[0] *100))

The amount of missing HADM_ID's ist tiny. We ignore these missing entrys.

In [ ]:
df_input_cv.head()

In [ ]:
sql = """select row_id from inputevents_cv """
df_test2 = pd.read_sql(sql, conn)

In [ ]:
df_test2.shape

In [ ]:
df_test2.nunique()